# Exploratory Analysis
Looking at Vaccination rates per city and county cross-referenced with demographic data to look for any correlations.
## Exploration Questions
Look at how the following correlate with vaccination rates.
- Median Income
- Parent Education Level
- Median House Price
- County Population

## Data being used
- Kindergarten Immunization records from Kaggle [link](https://www.kaggle.com/broach/california-kindergarten-immunization-rates)
- CA census estimates currated by the state [link](http://www.dof.ca.gov/Forecasting/Demographics/Estimates/E-5/)

# Imports

## Modules and Environment

In [80]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

import subprocess

from functools import reduce

get_ipython().run_line_magic('config', 'IPCompleter.greedy = True')

## Import data and basic cleaning of Data

### Immunization Records

#### Key for the table
- schoolType - Public/Private indicator
- COUNTY - name of county in CA
- SCHOOL - String label of school (not always consistent across years)
- school_code - Unique integer code for each school (consistent across years)
- n - Number of students
- nMMR - Number of students reporting complete MMR vaccination
- nDTP - Number of students reporting DTP vaccination
- nPolio - Number of students reporting Polio vaccination
- nPBE - Number of students reporting personal beliefs exemption*
- nPME - Number of students reporting permanent medical exemption
- year - Calendar year (2000:2014) where 2000=2000-2001 school year, 2001=2001-2002 school year, etc.

In [79]:
# Import immunization records
student_df = pd.read_csv('data/california-kindergarten-immunization-rates/StudentData.csv', sep=None, engine='python')
student_df.head()

,schoolType,COUNTY,SCHOOL,school_code,n,nMMR,nDTP,nPolio,nPBE,nPME,year
0,PRIVATE,ALAMEDA,A CHILD'S WORLD MONTE,7092463,12,12,11,9,0,0,2000
1,PRIVATE,ALAMEDA,ACAD,6972533,15,15,15,15,0,0,2000
2,PRIVATE,ALAMEDA,ACTS CHRISTIAN ACADEM,7082266,19,17,17,17,0,0,2009
3,PRIVATE,ALAMEDA,ACTS CHRISTIAN ACADEM,7082266,23,6,22,23,0,0,2010
4,PRIVATE,ALAMEDA,ACTS CHRISTIAN ACADEM,7082266,27,27,27,27,0,0,2006


In [3]:
# Narrow down to the years we have demo data
student_df = student_df[(student_df['year'] > 2009) & (student_df['year'] < 2015)]
student_df['year'].unique()

array([2010, 2012, 2011, 2014, 2013])

In [97]:
# Aggregate the student_df
student_df['exemptions'] = student_df['nPBE'] + student_df['nPME']
student_agg = student_df.groupby('COUNTY')['n', 'exemptions'].sum()

# Format it
student_agg = student_agg.reset_index()
student_agg = student_agg.rename(columns={'COUNTY': 'County'})
student_agg['County'] = student_agg['County'].str.title()

# Find the percent vacinated
student_agg['Percent Vaccinated'] = round((student_agg['n'] - student_agg['exemptions']) / student_agg['n'] * 100, 2)

student_agg.describe()
student_agg.head()

,County,n,exemptions,Percent Vaccinated
0,Alameda,301735,3694,98.78
1,Alpine,205,4,98.05
2,Amador,4937,183,96.29
3,Butte,40388,1835,95.46
4,Calaveras,6733,436,93.52


### Demographic Data

In [37]:
def filter_by_county(df):
    new_df = df.loc[df['Geography'].str.contains('County')]
    new_df = new_df.loc[~new_df['Geography'].str.contains('\(')]
    new_df['Geography'] = new_df['Geography'].str.replace(' County', '')
    new_df = new_df.rename(columns={'Geography': 'County'})
    return new_df

In [77]:
# Import Education Data
education_df = pd.read_excel('data/Web_ACS2014_10_Educ.xlsx', sheet_name='Educational Attainment', header=[4,5])

# Clean up the cols
education_df = education_df.rename(columns={'Unnamed: 0_level_0': 'Geography'})
education_df = education_df[["Geography", "Percent high school graduate or higher", "Percent bachelor's degree or higher"]]
education_df = education_df.drop(columns=['Margin of Error', 'Margin of Error.1', 'Summary Level', 'County', 'Place'], level=1)
education_df.columns = ['Geography',	'High School or Higher', 'Bachelors or Higher']

# Filter down to the county level
education_df = filter_by_county(education_df)

education_df.head()

,County,High School or Higher,Bachelors or Higher
1,Alameda,86.7,42.1
22,Alpine,91.4,30.5
28,Amador,88.4,20.5
47,Butte,88,25
78,Calaveras,92.5,21.1


In [39]:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 1 to 1567
Data columns (total 3 columns):
County                   58 non-null object
High School or Higher    58 non-null object
Bachelors or Higher      58 non-null object
dtypes: object(3)
memory usage: 1.8+ KB


In [91]:
# Import Health Insurance Data
health_ins_df = pd.read_excel('data/Web_ACS2014_10_HealthIns.xlsx', sheet_name='Health Insurance', header=[4,5])
health_ins_df = health_ins_df.drop(columns=['Estimate Margin of Error', 'Percent Margin of Error'], level=1)

health_ins_df = health_ins_df.iloc[:, :4]
health_ins_df.columns = ['Geography', 'Population', 'Number Insured', 'Percent Insured']
health_ins_df = filter_by_county(health_ins_df)

health_ins_df.shape

(58, 4)

In [89]:
# Import Income Data
income_df = pd.read_excel('data/Web_ACS2014_10_Inc-Pov-Emp.xlsx', sheet_name='Income', header=[3,4])
income_df = income_df[[('Unnamed: 0_level_0', 'Geography'), 
                                     ('Median household income (dollars)', 'Estimate'), 
                                     ('Per capita income (dollars)', 'Estimate')]]
income_df.columns = ['Geography', 'Median Income', 'Per capita income']
income_df = filter_by_county(income_df)

income_df.shape

(58, 3)

In [88]:
# Import Unemployment Data
unemployment_df_raw = pd.read_excel('data/Web_ACS2014_10_Inc-Pov-Emp.xlsx', sheet_name='Employment Status', header=[3,4,5,6])
unemployment_df = pd.DataFrame()
unemployment_df= unemployment_df_raw.iloc[:,[0, 17]]
unemployment_df.columns = ['Geography', 'Unemployment Percentage']

unemployment_df = filter_by_county(unemployment_df)
unemployment_df.shape

(58, 2)

### Merge data into one frame for analysis

In [122]:
dfs = [student_agg, education_df, health_ins_df, income_df, unemployment_df]

merged_df = reduce(lambda left, right: pd.merge(left, right, on='County'), dfs)

merged_df.loc[:, merged_df.columns != 'County'] = merged_df.loc[:, merged_df.columns != 'County'].apply(pd.to_numeric)
merged_df.head()
# convert_cols.head()


,County,n,exemptions,Percent Vaccinated,High School or Higher,Bachelors or Higher,Population,Number Insured,Percent Insured,Median Income,Per capita income,Unemployment Percentage
0,Alameda,301735,3694,98.78,86.7,42.1,1546984,1364761,88.2,73775,36439,6.3
1,Alpine,205,4,98.05,91.4,30.5,1202,935,77.8,61343,24375,3.2
2,Amador,4937,183,96.29,88.4,20.5,33018,29479,89.3,52964,27373,7.3
3,Butte,40388,1835,95.46,88.0,25.0,219426,186749,85.1,43165,24430,7.3
4,Calaveras,6733,436,93.52,92.5,21.1,44481,39604,89.0,54936,29296,5.8


In [123]:
# merged_df['High School or Higher'] = merged_df['High School or Higher'].astype('float64')

merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 0 to 57
Data columns (total 12 columns):
County                     58 non-null object
n                          58 non-null int64
exemptions                 58 non-null int64
Percent Vaccinated         58 non-null float64
High School or Higher      58 non-null float64
Bachelors or Higher        58 non-null float64
Population                 58 non-null int64
Number Insured             58 non-null int64
Percent Insured            58 non-null float64
Median Income              58 non-null int64
Per capita income          58 non-null int64
Unemployment Percentage    58 non-null float64
dtypes: float64(5), int64(6), object(1)
memory usage: 5.9+ KB


In [124]:
merged_df.describe()

,n,exemptions,Percent Vaccinated,High School or Higher,Bachelors or Higher,Population,Number Insured,Percent Insured,Median Income,Per capita income,Unemployment Percentage
count,5.800000e+01,58.000000,58.000000,58.000000,58.000000,5.800000e+01,5.800000e+01,58.000000,58.000000,58.000000,58.000000
mean,1.416535e+05,2764.758621,96.441379,83.201724,25.518966,6.474321e+05,5.393666e+05,84.543103,56034.362069,27604.344828,6.939655
std,3.113958e+05,4675.689953,2.888416,7.471337,10.503424,1.430881e+06,1.148325e+06,3.659120,14428.471908,8198.398555,1.656454
min,2.050000e+02,4.000000,85.400000,65.200000,12.500000,1.202000e+03,9.350000e+02,77.200000,35997.000000,16409.000000,3.200000
25%,7.175750e+03,286.250000,94.910000,78.325000,18.200000,4.610725e+04,4.088925e+04,81.275000,44717.250000,21754.250000,5.800000
50%,3.855450e+04,1089.500000,97.565000,85.650000,22.450000,1.787605e+05,1.565455e+05,84.800000,53302.000000,26190.000000,6.800000
75%,1.330208e+05,3218.250000,98.597500,88.775000,32.100000,6.490290e+05,5.440302e+05,87.600000,63946.750000,30913.750000,8.300000
max,2.144125e+06,26391.000000,99.600000,94.300000,54.800000,9.902803e+06,7.838038e+06,91.700000,93854.000000,58004.000000,10.600000
